In [9]:
import numpy as np
import cv2
from PIL import Image
import random
import timeit

from Algorithm import permute_rows, repermute_columns, repermute_rows, permute_columns, key_expansion, ciphering_block, key_update, deciphering_block, s, Arnold
from Image import blockshaped, From_blocks_to_arr_image
from Metrics import Entr, Hist, EQ, NPCR, UACI

In [10]:
def timer(function):
    def new_function(*args, **kwargs):
        start_time = timeit.default_timer()
        function(*args, **kwargs)
        elapsed = timeit.default_timer() - start_time
        print('Function "{name}" took {time} seconds to complete.'.format(name=function.__name__, time=elapsed))
    return new_function

In [11]:
@timer
def algo_confusion(arr, n):
    # constants
    r = 3.73421
    sum = 0
    for i in range(len(arr)):
        for j in range(len(arr)):
            sum = sum + arr[i,j]
    key = sum / n**2
    lkey = key - int(key)
    
    S_a = np.zeros(n, dtype=float)
    S_b = np.zeros(n, dtype=float)

    S_a[0] = lkey
    S_b[0] = 1 - lkey

    for i in range(1, n):
        S_a[i] =  r*S_a[i-1]*(1 - S_a[i-1])    
        S_b[i] =  r*S_b[i-1]*(1 - S_b[i-1])

    S_1n = np.concatenate((S_a[0:n//2], S_b[n//2:n]))
    S_2n = np.concatenate((S_b[0:n//2], S_a[n//2:n]))
    S_3n = np.concatenate((S_a[0:n//2], S_b[0:n//2]))
    S_4n = np.concatenate((S_b[n//2:n], S_a[n//2:n]))

    img_pi = permute_rows(arr, S_1n, n)
    img_pi = permute_rows(img_pi, S_2n, n)
    img_pi = permute_rows(img_pi, S_3n, n)
    img_pi = permute_rows(img_pi, S_4n, n)

    img_spi = permute_columns(img_pi, S_1n, n)
    img_spi = permute_columns(img_spi, S_2n, n)
    img_spi = permute_columns(img_spi, S_3n, n)
    img_spi = permute_columns(img_spi, S_4n, n)


    # step 4
    img_fpi = permute_rows(img_spi, S_a, n)
    img_fpi = permute_columns(img_fpi, S_b, n)


    # step 5
    L_1 = np.zeros((n**2)//2, dtype=float)
    L_2 = np.zeros((n**2)//2, dtype=float)
    LN_1 = np.zeros((n**2)//2, dtype=int)
    LN_2 = np.zeros((n**2)//2, dtype=int)

    L_1[0] = (1 - lkey)/3
    L_2[0] = (1 - lkey)/4

    for i in range(1, (n**2)//2):
        L_1[i] =  r*L_1[i-1]*(1 - L_1[i-1])    
        L_2[i] =  r*L_2[i-1]*(1 - L_2[i-1])

    for i in range((n**2)//2):
        LN_1[i] = (L_1[i] * (10**14)) % 256   
        LN_2[i] = (L_2[i] * (10**14)) % 256

    # step 6
    img_fpi_str = img_fpi.flatten()

    C = np.zeros(n**2, dtype=np.uint8)
    C[0] = LN_1[0] ^ LN_2[0] ^ img_fpi_str[0]
    C[1] = LN_2[0] ^ C[0] ^ img_fpi_str[1]

    for j in range(1, (n**2)//2):
        C[2*j] = LN_1[j] ^ C[2*j - 1] ^ img_fpi_str[2*j]
        C[2*j + 1] = LN_2[j] ^ C[2*j] ^ img_fpi_str[2*j + 1]
    arr = np.reshape(C, (-1, n))
    return arr

In [12]:
@timer
def algo_arnold(img):
    k1 = [1,32,3,234,15,156,7,8,9,40,1,2,3,43,5,64]
    k2 = [1,2,35,54,55,26,73,8,9,50,21,2,3,45,5,65]
    k3 = [1,27,3,24,25,6,7,8,9,0,13,2,23,4,5,64]
    k4 = [143,25,33,4,5,6,73,8,39,0,1,2,3,4,5,6]

    imgb = blockshaped(img, 16, 16)
    Nb = len(imgb)
    imgb_c = [0] * Nb
    imgb_d = [0] * Nb

    key_B = key_expansion(k1, k2, k3, k4)

    for i in range(Nb):
        B = imgb[i]
        B_C = ciphering_block(B, key_B, 8)
        imgb_c[i] = B_C
        key_B = key_update(key_B, B_C, B, k1, k2, k3, k4, 8)
        
    k = int(len(img[0]) / 16)

    img_enc = From_blocks_to_arr_image(imgb_c,Nb,k)
    return img_enc



In [17]:
time_img = cv2.imread('brain-2.jpg', 0)

time_res_1 = algo_arnold(time_img)
time_res_2 = algo_confusion(time_img, 256)


Function "algo_arnold" took 0.5199015919999965 seconds to complete.
Function "algo_confusion" took 22.928939997000043 seconds to complete.


In [6]:
'''
def generateSquareArrayOf256El():
    value_range = [ i for i in range(256)]
    arr1 = np.random.choice(value_range, (16,16))
    return arr1
    
def generatePairArray(diff, arr1):
    arr2 = np.bitwise_xor(diff, arr1)
    return arr2
    
    
    ==============================================================
    
diff_plain = generateSquareArrayOf256El()
n_pairs = 2
diff_cipher = []

for i in range(n_pairs):
    arr1 = generateSquareArrayOf256El()
    arr2 = generatePairArray(diff_plain, arr1)
    
    arr1_enc = algo(arr1)
    arr2_enc = algo(arr2)
    
    diff_cipher.append(np.bitwise_xor(arr1_enc, arr2_enc))
    
print(diff_cipher)
'''

'\ndef Entr_2(img,n):\n    arr_fl = np.array(img).flatten()\n\n    P_i = np.unique(arr_fl, return_counts=True)[1]/(img.shape[0]*img.shape[1])\n    print(len(P_i))\n    return (P_i * np.log2(P_i**(-1))).sum()\nEntr_2(time_img, 10)s\n\n\ndef generateSquareArrayOf256El():\n    value_range = [ i for i in range(256)]\n    arr1 = np.random.choice(value_range, (16,16))\n    return arr1\n    \ndef generatePairArray(diff, arr1):\n    arr2 = np.bitwise_xor(diff, arr1)\n    return arr2\n    \n    \n    ==============================================================\n    \ndiff_plain = generateSquareArrayOf256El()\nn_pairs = 2\ndiff_cipher = []\n\nfor i in range(n_pairs):\n    arr1 = generateSquareArrayOf256El()\n    arr2 = generatePairArray(diff_plain, arr1)\n    \n    arr1_enc = algo(arr1)\n    arr2_enc = algo(arr2)\n    \n    diff_cipher.append(np.bitwise_xor(arr1_enc, arr2_enc))\n    \nprint(diff_cipher)\n'

In [6]:
def NPCR_UACI_val(img_path, algo_type):
    img_1 = cv2.imread(img_path, 0)
    shape = img_1.shape
    NPCR_res_arr = []
    UACI_res_arr = []
    algo_types = ['Arnold_Vigenere', 'Confusion_Technique']
    
    if algo_type not in algo_types:
        raise ValueError("Invalid algo type. Expected one of: %s" % algo_types)
        
    for pix_i in range(shape[0]):
        for pix_j in range(shape[1]):
            
            img_2 = img_1.copy()
            img_2[pix_i][pix_j] = 255 - img_1[pix_i][pix_j]
            
            if algo_type == 'Arnold_Vigenere':
                img_enc_1 = algo_arnold(img_1)
                img_enc_2 = algo_arnold(img_2)
            else:
                img_enc_1 = algo_confusion(img_1, shape[0])
                img_enc_2 = algo_confusion(img_2, shape[0])
                
            NPCR_res_arr.append(NPCR(img_enc_1,img_enc_2))
            UACI_res_arr.append(UACI(img_enc_1,img_enc_2, 8))
            
    return [sum(NPCR_res_arr)/len(NPCR_res_arr), sum(UACI_res_arr)/len(UACI_res_arr)]

    

In [8]:
NPCR_UACI_val('station.jpg', 'Arnold_Vigenere')

[99.08890235457064, 49.76277768725218]

In [10]:
img = cv2.imread('station.jpg', 0)
img_e_1 = algo_confusion(img,304)
print(Entr(img_e_1))
img_e_2 = algo_arnold(img)
print(Entr(img_e_2))

7.998336184950415
7.997892593677403
